In [1]:
%load_ext autoreload
%autoreload 2
%cd ..

/cis/home/dpacker/my_documents/vote-counts


In [2]:
import pandas as pd
from pathlib import Path
import numpyro
from numpyro import distributions as dist
from numpyro.infer import NUTS, MCMC, Predictive
from jax import random, numpy as jnp
import matplotlib.pyplot as plt
import seaborn as sns
import os
from numpyro.contrib.funsor import config_enumerate, infer_discrete
from src import preprocessing

os.environ['CUDA_VISIBLE_DEVICES'] = '3'

path_data = Path("data")
path_raw_data = path_data / "raw_data"

In [129]:
raw_data = pd.read_csv(path_raw_data / "Local_2018.csv")
data = preprocessing.preprocess(raw_data)

/tmp/ipykernel_981314/80380610.py:1: DtypeWarning: Columns (2,3,11) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv(path_raw_data / "Local_2018.csv")


In [130]:
# election_df = data.groupby(["precinct", "office", "candidate"]).sum("votes")
ohio_data = data[data["state"] == "OHIO"]
ohio_data.groupby(["state", "office", "candidate"]).sum(numeric_only=True).groupby(["state", "office", "precincts", "candidate"]).sum()

votes  \
state office                                             precincts candidate                     
OHIO  CLERK OF THE COURT OF COMMON PLEAS - UNEXPIRED ... 25        SANDRA I WILSON        7097   
                                                         39        DEBORAH A MYERS       13203   
                                                                   MICHAEL R ZICKEFOOSE   5190   
                                                         146       AJ WILLIAMS           41921   
                                                                   CYNDI PAUWELS         25148   
...                                                                                        ...   
      PROSECUTING ATTORNEY - UNEXPIRED TERM ENDING 01... 65        JEFF RATLIFF           7978   
                                                                   RAY GROGAN            12648   
                                                         77        RACHEL DAEHLER         8836   
                                                                   SHANE TIEMAN          14893   
      SHERIFF - UNEXPIRED TERM ENDING 01/03/2021         88        MATTHEW B TREGLIA     29801   

                                                                                         county_fips  \
state office                                             precincts candidate                           
OHIO  CLERK OF THE COURT OF COMMON PLEAS - UNEXPIRED ... 25        SANDRA I WILSON            976175   
                                                         39        DEBORAH A MYERS           1521195   
                                                                   MICHAEL R ZICKEFOOSE      1521195   
                                                         146       AJ WILLIAMS               5702322   
                                                                   CYNDI PAUWELS             5702322   
...                                                                                              ...   
      PROSECUTING ATTORNEY - UNEXPIRED TERM ENDING 01... 65        JEFF RATLIFF              2541565   
                                                                   RAY GROGAN                2541565   
                                                         77        RACHEL DAEHLER            3014165   
                                                                   SHANE TIEMAN              3014165   
      SHERIFF - UNEXPIRED TERM ENDING 01/03/2021         88        MATTHEW B TREGLIA         3432264   

                                                                                         jurisdiction_fips  \
state office                                             precincts candidate                                 
OHIO  CLERK OF THE COURT OF COMMON PLEAS - UNEXPIRED ... 25        SANDRA I WILSON                  976175   
                                                         39        DEBORAH A MYERS                 1521195   
                                                                   MICHAEL R ZICKEFOOSE            1521195   
                                                         146       AJ WILLIAMS                     5702322   
                                                                   CYNDI PAUWELS                   5702322   
...                                                                                                    ...   
      PROSECUTING ATTORNEY - UNEXPIRED TERM ENDING 01... 65        JEFF RATLIFF                    2541565   
                                                                   RAY GROGAN                      2541565   
                                                         77        RACHEL DAEHLER                  3014165   
                                                                   SHANE TIEMAN                    3014165   
      SHERIFF - UNEXPIRED TERM ENDING 01/03/2021         88        MATTHEW B TREGLIA               3432264   

                              

In [131]:
grouped = data.groupby(['state', 'candidate', 'office']).count()

In [132]:
grouped = grouped.reset_index()
grouped["offices_sought"] = 1
grouped_offices_sought = grouped.groupby("candidate").sum()["offices_sought"]

In [133]:
grouped_offices_sought[grouped_offices_sought != 1]

candidate
AGAINST                       6
AGAINST BONDS                 4
ALAN KNAPHUS                  2
ALLEN                         2
ALLEN ANDERSON                2
                           ... 
WOLFF                         2
WOODROW WRIGHT                2
WRIGHT                        3
YES                         499
YES-LOCAL AMENDMENT NO 1      2
Name: offices_sought, Length: 419, dtype: int64

In [138]:
election_details = "ALAN KNAPHUS", "TOWNSHIP CLERK - WARREN"

query_data = lambda df, candidate, office: df[
    (df["candidate"] == candidate) & (df["office"] == office)
]
get_neighborhood = lambda df, office, precinct: df[
    (df["office"] == office) & (df["precinct"] == precinct)
].index


def expand_graph(indices):
    new_indices = pd.Index([])
    for index in indices:
        rows = data.loc[index]

        new_indices = new_indices.union(get_neighborhood(data, row))
    return new_indices


rows = query_data(data, *election_details)
election_indices = [6]

In [161]:
# data.loc[expand_graph([0])[1]]['candidate']


new_indices = pd.Index([])
rows = data.loc[election_indices, ["office", "candidate"]]
office = rows.iloc[0, 0]
candidates = rows["candidate"].unique()
precincts = pd.concat([query_data(data, candidate, office)["precinct"] for candidate in candidates]).unique()
for precinct in precincts[:-2]:
  new_indices = new_indices.union(get_neighborhood(data, office, precinct))
# new_indices

In [163]:
data.loc[new_indices, "candidate"]

# precincts

6      DEBRA HILL
10     DEBRA HILL
42     DEBRA HILL
46     DEBRA HILL
50     DEBRA HILL
54     DEBRA HILL
58     DEBRA HILL
62     DEBRA HILL
122    DEBRA HILL
168    DEBRA HILL
172    DEBRA HILL
179    DEBRA HILL
207    DEBRA HILL
262    DEBRA HILL
314    DEBRA HILL
318    DEBRA HILL
322    DEBRA HILL
Name: candidate, dtype: object

In [109]:
data.query("county_name == 'PERRY'").query("office == 'COUNTY SHERIFF'")

,precinct,office,party_detailed,party_simplified,mode,votes,county_name,county_fips,jurisdiction_name,jurisdiction_fips,...,state,special,writein,state_po,state_fips,state_cen,state_ic,date,readme_check,precincts
0,SEVERE,COUNTY SHERIFF,NaN,NaN,TOTAL,0,PERRY,1105,PERRY,1105,...,ALABAMA,False,False,AL,1,63,41,2018-11-06,False,1
50059,ARMORY,COUNTY SHERIFF,NaN,NaN,TOTAL,0,PERRY,1105,PERRY,1105,...,ALABAMA,False,False,AL,1,63,41,2018-11-06,False,1
50060,ARMORY,COUNTY SHERIFF,NaN,NaN,TOTAL,256,PERRY,1105,PERRY,1105,...,ALABAMA,False,False,AL,1,63,41,2018-11-06,False,1
50062,ARMORY,COUNTY SHERIFF,DEMOCRAT,DEMOCRAT,TOTAL,1110,PERRY,1105,PERRY,1105,...,ALABAMA,False,False,AL,1,63,41,2018-11-06,False,1
51172,COMMUNITY LIFE BLDG,COUNTY SHERIFF,NaN,NaN,TOTAL,0,PERRY,1105,PERRY,1105,...,ALABAMA,False,False,AL,1,63,41,2018-11-06,False,1
51173,COMMUNITY LIFE BLDG,COUNTY SHERIFF,NaN,NaN,TOTAL,22,PERRY,1105,PERRY,1105,...,ALABAMA,False,False,AL,1,63,41,2018-11-06,False,1
51175,COMMUNITY LIFE BLDG,COUNTY SHERIFF,DEMOCRAT,DEMOCRAT,TOTAL,782,PERRY,1105,PERRY,1105,...,ALABAMA,False,False,AL,1,63,41,2018-11-06,False,1
51299,COUNTY FLOATING,COUNTY SHERIFF,NaN,NaN,ABSENTEE,0,PERRY,1105,PERRY,1105,...,ALABAMA,False,False,AL,1,63,41,2018-11-06,False,1
51363,COUNTY FLOATING,COUNTY SHERIFF,NaN,NaN,PROVISIONAL,0,PERRY,1105,PERRY,1105,...,ALABAMA,False,False,AL,1,63,41,2018-11-06,False,1
51414,COUNTY FLOATING,COUNTY SHERIFF,NaN,NaN,ABSENTEE,26,PERRY,1105,PERRY,1105,...,ALABAMA,False,False,AL,1,63,41,2018-11-06,False,1


In [63]:
get_neighborhood(data, rows.iloc[0])

,precinct,office,party_detailed,party_simplified,mode,votes,county_name,county_fips,jurisdiction_name,jurisdiction_fips,...,state,special,writein,state_po,state_fips,state_cen,state_ic,date,readme_check,precincts
571927,WARREN/MCCALLSBURG,TOWNSHIP CLERK - WARREN,NaN,NaN,ABSENTEE,1,STORY,19169,STORY,19169,...,IOWA,False,False,IA,19,42,31,2018-11-06,False,1
571928,WARREN/MCCALLSBURG,TOWNSHIP CLERK - WARREN,NaN,NaN,ELECTION DAY,0,STORY,19169,STORY,19169,...,IOWA,False,False,IA,19,42,31,2018-11-06,False,1
571929,WARREN/MCCALLSBURG,TOWNSHIP CLERK - WARREN,NaN,NaN,ABSENTEE,1,STORY,19169,STORY,19169,...,IOWA,False,False,IA,19,42,31,2018-11-06,False,1
571930,WARREN/MCCALLSBURG,TOWNSHIP CLERK - WARREN,NaN,NaN,ELECTION DAY,0,STORY,19169,STORY,19169,...,IOWA,False,False,IA,19,42,31,2018-11-06,False,1
571931,WARREN/MCCALLSBURG,TOWNSHIP CLERK - WARREN,NaN,NaN,ABSENTEE,1,STORY,19169,STORY,19169,...,IOWA,False,False,IA,19,42,31,2018-11-06,False,1
571932,WARREN/MCCALLSBURG,TOWNSHIP CLERK - WARREN,NaN,NaN,ELECTION DAY,0,STORY,19169,STORY,19169,...,IOWA,False,False,IA,19,42,31,2018-11-06,False,1
571933,WARREN/MCCALLSBURG,TOWNSHIP CLERK - WARREN,NaN,NaN,ABSENTEE,0,STORY,19169,STORY,19169,...,IOWA,False,False,IA,19,42,31,2018-11-06,False,1
571934,WARREN/MCCALLSBURG,TOWNSHIP CLERK - WARREN,NaN,NaN,ELECTION DAY,1,STORY,19169,STORY,19169,...,IOWA,False,False,IA,19,42,31,2018-11-06,False,1
571935,WARREN/MCCALLSBURG,TOWNSHIP CLERK - WARREN,NaN,NaN,ABSENTEE,3,STORY,19169,STORY,19169,...,IOWA,False,False,IA,19,42,31,2018-11-06,False,1
571936,WARREN/MCCALLSBURG,TOWNSHIP CLERK - WARREN,NaN,NaN,ELECTION DAY,2,STORY,19169,STORY,19169,...,IOWA,False,False,IA,19,42,31,2018-11-06,False,1
